# Encontrando una política para jugar el 21 (con repartidor)

Esta libreta utiliza dos algoritmos de programación dinámica para encontrar una política que permita jugar al 21.

Las reglas para este 21 son especiales:
- El mazo es infinito. De esta manera las 13 cartas siempre tienen las mismas probabilidades de aparecer
- El repartidor no realiza movimientos hasta que el jugador decida parar
- Quien llegue a 4 cartas sin pasarse automáticamente gana
- El repartidor siempre pide una carta más si tiene 16 o menos. En otro caso, siempre para

Con las reglas explicadas, definiremos una estructura para procesos de decisión markoviana.

In [1]:
struct MDP
    states
    actions
    ρ
    reward
    final_s
end

## Funciones para hallar la política

### Valor de política

Primero implementaremos una función que nos permitirá calcular el valor para cada estado de nuestro Proceso de Decisión de Markov con una política $\pi$ y con factor de descuento $\gamma$.

Esta función calcula iterativamente el valor para cada estado hasta que los resultados convergen.

In [18]:
"""
    policy_value(mdp::MDP, pol_π::Dict, γ::Float64)

Computes the value for every state in the MDP by using pol_π and discount γ.
"""
function policy_value(mdp::MDP, pol_π::Dict, γ::Float64)
    v = Dict(s => 0.0 for s ∈ mdp.states)
    
    has_converged = false
    while !has_converged
        has_converged = true
    
        for s ∈ mdp.states
            temp = sum([mdp.ρ(s, pol_π[s], n_s) * (mdp.reward(s, pol_π[s], n_s) + γ*v[n_s]) for n_s ∈ keys(v)])
                    
            if temp != v[s]
                has_converged = false
            end
            
            v[s] = temp
        end           
    end
            
    return v
end

policy_value

### Iteración de política

Con la función anterior podemos programar ahora una función que encuentre una política óptima para un Proceso de Decisión de Markov.

In [19]:
"""
    policy_iteration(mdp::MDP, γ)

Finds an optimal policy for the MDP using discount factor γ.
"""
function policy_iteration(mdp::MDP, γ)
    pol_π = Dict(s => rand(mdp.actions) for s ∈ mdp.states)
    
    is_optimal = false
    while !is_optimal
        v = policy_value(mdp, pol_π, γ)
        
        is_optimal = true
        
        for s ∈ keys(v)
            for a ∈ mdp.actions
                temp = sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ*v[s]) for n_s ∈ keys(v)])
                
                if temp < v[s]
                    is_optimal = false
                    pol_π[s] = a
                end
            end
        end
    end
    
    return pol_π
end

policy_iteration

### Iteración de valor

Enseguida viene una implementación de un algoritmo iterativo para encontrar una buena política, todo en un único bloque de código.

In [4]:
"""
    iter_value(mdp::MDP, γ::Float64)

Iteratively computes the value function of a Markov Decision Process using
discount rate γ and then returns the optimal policy π associated with it.
"""
function iter_value(mdp::MDP, γ::Float64)
    v = Dict(s => 0.0 for s ∈ mdp.states)
    v_p = Dict(s => 0.0 for s ∈ mdp.states)
    
    has_converged = false
    
    while !has_converged
        for s ∈ keys(v)
            if s ∈ mdp.final_s
                v_p[s] = mdp.reward(s, nothing, s)
            else
                v_p[s] = maximum([sum([mdp.ρ(s, a, n_s) * (mdp.reward(s, a, n_s) + γ * v[n_s])
                                            for n_s ∈ mdp.states])
                                        for a ∈ mdp.actions])
            end

            has_converged = true
                            
            for s ∈ keys(v)
                if v_p[s] > v[s]
                    v[s] = v_p[s]
                    has_converged = false
                end
            end
            
            if has_converged
                break
            end
        end
    end
    
    pol_π = Dict(s => mdp.actions[1] for s ∈ mdp.states)
    
    for s ∈ keys(v)
        actions_value = Dict(a => sum([mdp.ρ(s, a, n_s) * v[n_s] for n_s ∈ mdp.states])
                            for a ∈ mdp.actions)
                                
        pol_π[s] = findmax(actions_value)[2]
    end
    
    return pol_π
end

iter_value

## Definiendo el proceso de decisión de Markov

Una vez que tenemos las funciones que nos permitirán encontrar una política óptima para el 21, ahora ocupamos definir un modelo que se ajuste a estos.
En las siguientes celdas viene el código necesario para definir un proceso de decisión de Markov que represente el juego y que sea compatible con ``policy_value``, ``policy_iteration``, ``iter_value``.

### Estados del 21

Primero definiremos los estados del juego como un arreglo de 5 números enteros:

$$(C_p, S_p, C_d, S_d, f)$$

Donde $S_p$ es la suma total de la cantidad $C_p$ de cartas que tiene el jugador. $S_d$ es la suma total de la cantidad $C_d$ de cartas que tiene la casa. $f$ nos indica si el estado es terminal o no: 0 si no lo es, 1 si lo es.

Calculamos cada estado del juego de manera progresiva: primero todas las posibilidades cuando el jugador tiene 2 cartas, luego cuando tiene 3, y finalmente cuando tiene 4.

In [5]:
states = []

# 2 cartas del jugador.
for c ∈ 1:4
    fin = if c < 3 10*c + 1 else 26 end
    
    for j ∈ c:fin
        fin_p = if c == 1 21 else 20 end
        
        for i ∈ 2:fin_p
            push!(states, [2, i, c, j, 0])
        end
    end
end

# 3 cartas del jugador.
for c ∈ 1:4
    fin = if c < 3 10*c + 1 else 26 end
    
    for j ∈ c:fin
        fin_p = if c == 1 30 else 20 end
        
        for i ∈ 3:fin_p
            push!(states, [3, i, c, j, 0])
        end
    end
end

# 4 cartas del jugador.  
for j ∈ 1:11
    for i ∈ 4:30
        push!(states, [4, i, 1, j, 0])
    end
end

for s ∈ states
    if s[2] >= 21 || s[4] >= 17 || s[1] == 4 || s[3] == 4
        s[5] = 1
    end
end

states

3304-element Array{Any,1}:
 [2, 2, 1, 1, 0]  
 [2, 3, 1, 1, 0]  
 [2, 4, 1, 1, 0]  
 [2, 5, 1, 1, 0]  
 [2, 6, 1, 1, 0]  
 [2, 7, 1, 1, 0]  
 [2, 8, 1, 1, 0]  
 [2, 9, 1, 1, 0]  
 [2, 10, 1, 1, 0] 
 [2, 11, 1, 1, 0] 
 [2, 12, 1, 1, 0] 
 [2, 13, 1, 1, 0] 
 [2, 14, 1, 1, 0] 
 ⋮                
 [4, 19, 1, 11, 1]
 [4, 20, 1, 11, 1]
 [4, 21, 1, 11, 1]
 [4, 22, 1, 11, 1]
 [4, 23, 1, 11, 1]
 [4, 24, 1, 11, 1]
 [4, 25, 1, 11, 1]
 [4, 26, 1, 11, 1]
 [4, 27, 1, 11, 1]
 [4, 28, 1, 11, 1]
 [4, 29, 1, 11, 1]
 [4, 30, 1, 11, 1]

Podemos revisar cuantos estados fueron generados con la siguiente celda.

In [6]:
println("Cantidad de estados:", size(states))

Cantidad de estados:(3304,)


Esos son bastantes estados, pero ni modo, podría ser mucho peor. La cantidad de estados necesarios para describir un entorno incrementa rápidamente para problemas más complejos.

### Estados finales

Una vez que tenemos todos los estados del 21, agregamos todos los estados finales a un único lugar.

In [7]:
final_states = []
for s ∈ states
    if s[5] == 1
        push!(final_states, s)
    end
end

final_states

1824-element Array{Any,1}:
 [2, 21, 1, 1, 1] 
 [2, 21, 1, 2, 1] 
 [2, 21, 1, 3, 1] 
 [2, 21, 1, 4, 1] 
 [2, 21, 1, 5, 1] 
 [2, 21, 1, 6, 1] 
 [2, 21, 1, 7, 1] 
 [2, 21, 1, 8, 1] 
 [2, 21, 1, 9, 1] 
 [2, 21, 1, 10, 1]
 [2, 21, 1, 11, 1]
 [2, 2, 2, 17, 1] 
 [2, 3, 2, 17, 1] 
 ⋮                
 [4, 19, 1, 11, 1]
 [4, 20, 1, 11, 1]
 [4, 21, 1, 11, 1]
 [4, 22, 1, 11, 1]
 [4, 23, 1, 11, 1]
 [4, 24, 1, 11, 1]
 [4, 25, 1, 11, 1]
 [4, 26, 1, 11, 1]
 [4, 27, 1, 11, 1]
 [4, 28, 1, 11, 1]
 [4, 29, 1, 11, 1]
 [4, 30, 1, 11, 1]

### Acciones

Ahora las acciones. Esta parte es fácil, solo hay dos posibles acciones en todo momento. Esta libreta usa el vocabulario usado en los casinos.

In [8]:
actions = ["hit", "stand"]

2-element Array{String,1}:
 "hit"  
 "stand"

### Probabilidad de transición

Seguimos con la declaración de la función que calcula la probabilidad de transición entre estados.

In [16]:
function ρ(s, a, n_s)
    # No permitimos que se salga de un estado final.
    if s[5] == 1
        return 0
    end
    
    if a == "stand"
        # Nos ahorramos el trabajo si es una transición al mismo estado.
        if s == n_s
            return 1
        end
        
        # No permitimos que juegue después de terminar.
        if s[1] != n_s[1]
            return 0
        end
        
        diff_score = n_s[4] - s[4]
        has_one_more = n_s[3] == s[3] + 1
        same_player_score = s[2] == n_s[2]
        
        if has_one_more && (1 <= diff_score <= 9 || diff_score == 11) && same_player_score
            return 1/13
        elseif has_one_more && diff_score == 10 && same_player_score
            return 4/13
        end
    else
        # No permitimos que juegue después de terminar.
        if s[3] != 1 || n_s[3] != 1
            return 0
        end
        
        diff_score = n_s[2] - s[2]
        has_one_more = n_s[1] == s[1] + 1
        same_dealer_score = s[4] == n_s[4]
        
        if has_one_more && (1 <= diff_score <= 9 || diff_score == 11) && same_dealer_score
            return 1/13
        elseif has_one_more && diff_score == 10 && same_dealer_score
            return 4/13
        end
    end
    
    return 0
end

ρ (generic function with 1 method)

### Recompensa

Completamos el modelo calculando la recompensa para cada estado. Aquí no hay ganancia ni pérdida a menos que el juego se acabe.

In [10]:
function reward(s, a, n_s)
    if n_s[5] == 1
        if n_s[2] <= 21
            if n_s[2] == 21 || n_s[1] == 4 || n_s[2] >= n_s[4] || n_s[4] > 21
                return 1
            else
                return -1
            end
        else
            return -1
        end
    else
        return 0
    end
end

reward (generic function with 1 method)

## Resolviendo el problema

Como tenemos todos los preparativos listos, podemos crear un nuevo proceso de decisión de Markov que modele este 21.

In [11]:
twenty_one = MDP(states, actions, ρ, reward, final_states)

MDP(Any[[2, 2, 1, 1, 0], [2, 3, 1, 1, 0], [2, 4, 1, 1, 0], [2, 5, 1, 1, 0], [2, 6, 1, 1, 0], [2, 7, 1, 1, 0], [2, 8, 1, 1, 0], [2, 9, 1, 1, 0], [2, 10, 1, 1, 0], [2, 11, 1, 1, 0]  …  [4, 21, 1, 11, 1], [4, 22, 1, 11, 1], [4, 23, 1, 11, 1], [4, 24, 1, 11, 1], [4, 25, 1, 11, 1], [4, 26, 1, 11, 1], [4, 27, 1, 11, 1], [4, 28, 1, 11, 1], [4, 29, 1, 11, 1], [4, 30, 1, 11, 1]], ["hit", "stand"], ρ, reward, Any[[2, 21, 1, 1, 1], [2, 21, 1, 2, 1], [2, 21, 1, 3, 1], [2, 21, 1, 4, 1], [2, 21, 1, 5, 1], [2, 21, 1, 6, 1], [2, 21, 1, 7, 1], [2, 21, 1, 8, 1], [2, 21, 1, 9, 1], [2, 21, 1, 10, 1]  …  [4, 21, 1, 11, 1], [4, 22, 1, 11, 1], [4, 23, 1, 11, 1], [4, 24, 1, 11, 1], [4, 25, 1, 11, 1], [4, 26, 1, 11, 1], [4, 27, 1, 11, 1], [4, 28, 1, 11, 1], [4, 29, 1, 11, 1], [4, 30, 1, 11, 1]])

### Iteración de política

Y con esto, ahora podemos probar si nuestros algoritmos realmente cumplen su tarea. Desafortunadamente, este MDP no funciona bien con las siguientes funciones. Los valores divergen. Estas pruebas se quedan en esta libreta por si te interesa depurar el modelo y mejorarlo.

In [ ]:
γ = 0.9
pol_π_1 = policy_iteration(twenty_one, γ)

Ahora revisamos el valor para la política $\pi_1$ y los contenidos de la misma.

In [ ]:
policy_value(twenty_one, pol_π_1, γ, 0.5)

In [ ]:
for s in states
    println(s, ": ", pol_π_1[s])
end

### Iteración de valor

Por último, probamos el algoritmo de iteración de valor y examinamos el valor para la política $\pi_2$.

In [21]:
γ = 0.9
pol_π_2 = iter_value(twenty_one, γ)

Dict{Array{Int64,1},String} with 3304 entries:
  [3, 11, 4, 17, 1] => "stand"
  [2, 3, 3, 6, 0]   => "stand"
  [3, 6, 1, 10, 0]  => "stand"
  [3, 10, 3, 26, 1] => "stand"
  [2, 21, 1, 3, 1]  => "stand"
  [2, 15, 1, 5, 0]  => "stand"
  [4, 26, 1, 1, 1]  => "stand"
  [2, 12, 1, 10, 0] => "stand"
  [2, 12, 2, 15, 0] => "stand"
  [4, 23, 1, 8, 1]  => "stand"
  [2, 7, 3, 19, 1]  => "stand"
  [3, 9, 3, 20, 1]  => "stand"
  [2, 7, 2, 8, 0]   => "stand"
  [2, 3, 1, 6, 0]   => "stand"
  [3, 18, 1, 3, 0]  => "stand"
  [2, 17, 4, 21, 1] => "stand"
  [2, 4, 4, 22, 1]  => "stand"
  [3, 15, 3, 10, 0] => "stand"
  [2, 3, 4, 16, 1]  => "stand"
  [3, 11, 2, 10, 0] => "stand"
  [2, 19, 3, 15, 0] => "stand"
  [3, 28, 1, 5, 1]  => "stand"
  [4, 16, 1, 11, 1] => "stand"
  [2, 14, 4, 4, 1]  => "stand"
  [3, 12, 4, 24, 1] => "stand"
  ⋮                 => ⋮

In [ ]:
policy_value(twenty_one, pol_π_2, γ)

In [ ]:
for s in states
    println(s, ": ", pol_π_2[s])
end